In [1]:
import pandas as pd
import joblib

In [2]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

df_train = df_train.drop(columns=['FG%','3P%','FT%'])
df_test = df_test.drop(columns=['FG%','3P%','FT%'])

# df_train = df_train.drop(columns=['FGM','FGA','3P Made','3PA','FTM','FTA','BLK'])
# df_test = df_test.drop(columns=['FGM','FGA','3P Made','3PA','FTM','FTA','BLK'])

In [3]:
def getAbsolute(df, name):
    df = df.abs()
    joblib.dump(df, "../data/processed/abs_"+name)
    return df

In [4]:
df_train = getAbsolute(df_train, "train")
df_test = getAbsolute(df_test, "test")

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df_train.iloc[:,2:-1]
X['MPG'] = X.apply(lambda row: row['MIN'] / row['GP'], axis = 1)
X['FG%'] = X.apply(lambda row: row['FGM'] / row['FGA'] if row['FGA'] > 0 else 0, axis = 1)
# X['FG%_Rank'] = X.apply(lambda row: 1 if row['FG%'] >= 0.5 else 0, axis = 1)
X['3P%'] = X.apply(lambda row: row['3P Made'] / row['3PA'] if row['3PA'] > 0 else 0, axis = 1)
# X['3P%_Rank'] = X.apply(lambda row: 1 if row['3P%'] >= 0.5 else 0, axis = 1)
X['FT%'] = X.apply(lambda row: row['FTM'] / row['FTA'] if row['FTA'] > 0 else 0, axis = 1)
# X['FT%_Rank'] = X.apply(lambda row: 1 if row['FT%'] >= 0.5 else 0, axis = 1)
X['GP_small'] = X.apply(lambda row: 1 if row['GP'] < 50 else 0, axis = 1)
X['GP_medium'] = X.apply(lambda row: 1 if (row['GP'] >= 50 and row['GP'] < 100) else 0, axis = 1)
X['MIN_small'] = X.apply(lambda row: 1 if row['MIN'] <= 20 else 0, axis = 1)
X['MIN_medium'] = X.apply(lambda row: 1 if (row['MIN'] > 20  and row['MIN'] <= 40) else 0, axis = 1)
X['NotActive'] = X.apply(lambda row: 1 if (row['FGA']*row['3PA']*row['FTA']*row['AST']*row['BLK']*row['TOV'] == 0) else 0, axis = 1)
# X['GP_large'] = X.apply(lambda row: 1 if (row['GP'] > 100) else 0, axis = 1)
# X = X.drop(columns=['MIN','GP','FG%','3P%','FT%'])
# X = X.drop(columns=['FG%','3P%','FT%'])
X

,GP,MIN,PTS,FGM,FGA,3P Made,3PA,FTM,FTA,OREB,...,TOV,MPG,FG%,3P%,FT%,GP_small,GP_medium,MIN_small,MIN_medium,NotActive
0,80,24.3,7.8,3.0,6.4,0.1,0.3,2.0,2.9,2.2,...,1.6,0.303750,0.468750,0.333333,0.689655,0,1,0,1,0
1,75,21.8,10.5,4.2,7.9,0.3,1.0,2.4,3.6,3.6,...,1.4,0.290667,0.531646,0.300000,0.666667,0,1,0,1,0
2,85,19.1,4.5,1.9,4.5,0.4,1.2,0.4,0.6,0.6,...,0.6,0.224706,0.422222,0.333333,0.666667,0,1,1,0,0
3,63,19.1,8.2,3.5,6.7,0.3,0.8,0.9,1.5,0.8,...,1.9,0.303175,0.522388,0.375000,0.600000,0,1,1,0,0
4,63,17.8,3.7,1.7,3.4,0.5,1.4,0.2,0.5,2.4,...,0.7,0.282540,0.500000,0.357143,0.400000,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,32,9.2,1.8,0.7,1.8,0.1,0.2,0.4,0.6,0.3,...,0.4,0.287500,0.388889,0.500000,0.666667,1,0,1,0,0
7996,54,6.0,1.8,0.7,1.4,0.1,0.1,0.2,0.4,1.0,...,0.3,0.111111,0.500000,1.000000,0.500000,0,1,1,0,0
7997,85,28.2,10.7,4.0,9.0,0.2,0.6,2.8,3.9,1.0,...,1.8,0.331765,0.444444,0.333333,0.717949,0,1,0,1,0
7998,39,7.7,2.5,1.0,2.3,0.3,0.5,0.6,0.7,0.4,...,0.5,0.197436,0.434783,0.600000,0.857143,1,0,1,0,0


In [7]:
y = df_train.iloc[:,-1:]
y

,TARGET_5Yrs
0,1
1,1
2,1
3,1
4,1
...,...
7995,1
7996,1
7997,1
7998,1


In [8]:
df_train.TARGET_5Yrs.value_counts()

1    6669
0    1331
Name: TARGET_5Yrs, dtype: int64

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=8)
y_train.value_counts()

TARGET_5Yrs
1              5326
0              1074
dtype: int64

In [10]:
y_val.value_counts()

TARGET_5Yrs
1              1343
0               257
dtype: int64

In [11]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler().fit(X_train)
X_train = sc.transform(X_train)
X_val = sc.transform(X_val)

In [13]:
X_test = df_test.iloc[:,2:]
X_test['MPG'] = X_test.apply(lambda row: row['MIN'] / row['GP'], axis = 1)
X_test['FG%'] = X_test.apply(lambda row: row['FGM'] / row['FGA'] if row['FGA'] > 0 else 0, axis = 1)
X_test['3P%'] = X_test.apply(lambda row: row['3P Made'] / row['3PA'] if row['3PA'] > 0 else 0, axis = 1)
X_test['FT%'] = X_test.apply(lambda row: row['FTM'] / row['FTA'] if row['FTA'] > 0 else 0, axis = 1)
X_test['GP_small'] = X_test.apply(lambda row: 1 if row['GP'] < 50 else 0, axis = 1)
X_test['GP_medium'] = X_test.apply(lambda row: 1 if (row['GP'] >= 50 and row['GP'] < 100) else 0, axis = 1)
# X_test['GP_large'] = X_test.apply(lambda row: 1 if (row['GP'] > 100) else 0, axis = 1)
X_test['MIN_small'] = X_test.apply(lambda row: 1 if row['MIN'] <= 20 else 0, axis = 1)
X_test['MIN_medium'] = X_test.apply(lambda row: 1 if (row['MIN'] > 20  and row['MIN'] <= 40) else 0, axis = 1)
X_test['NotActive'] = X_test.apply(lambda row: 1 if (row['FGA']*row['3PA']*row['FTA']*row['AST']*row['BLK']*row['TOV'] == 0) else 0, axis = 1)
# X_test = X_test.drop(columns=['MIN','GP'])
X_test = sc.transform(X_test)

In [14]:
X_test_id = df_test.iloc[:,1:2]
X_test_id

,Id
0,0
1,1
2,2
3,3
4,4
...,...
3794,3794
3795,3795
3796,3796
3797,3797


In [15]:
joblib.dump(X_train, "../data/processed/X_train")
joblib.dump(y_train, "../data/processed/y_train")
joblib.dump(X_val, "../data/processed/X_val")
joblib.dump(y_val, "../data/processed/y_val")
joblib.dump(X_test, "../data/processed/X_test")
joblib.dump(X_test_id, "../data/processed/X_test_id")

['../data/processed/X_test_id']